In [53]:
import os
DATA_DIR = '../data'

# Experiment Config
DF_NAME = 'MathQA'
DIFFICULTY = 'easy'
NUM_OF_SAMPLES = 500
NUM_OF_COT = 40
storage_dir = os.path.join(DATA_DIR, f'Evaluation_CoTs/gpt-3.5-turbo-0125')
file_path = os.path.join(storage_dir, f'{DF_NAME}_{DIFFICULTY}.csv')

In [54]:
import pandas as pd
df = pd.read_csv(file_path)
df

,Name,Category,Question,Correct Answer,CoT_0,Final Answer_0,Instruction Violation_0,CoT_1,Final Answer_1,Instruction Violation_1,...,Instruction Violation_36,CoT_37,Final Answer_37,Instruction Violation_37,CoT_38,Final Answer_38,Instruction Violation_38,CoT_39,Final Answer_39,Instruction Violation_39
0,MathQA_dev,Math,a multiple choice test consists of 4 questions...,c,\nStep 1: To find the number of ways the test ...,c,"[(0, 0)]",\nStep 1: To find the number of ways the test ...,c,"[(0, 0)]",...,"[(0, 0)]",\nStep 1: To find the number of ways the test ...,c,"[(0, 0)]",\nStep 1: To find the number of ways the test ...,c,"[(0, 0)]",\nStep 1: To find the number of ways the test ...,c,"[(0, 0)]"
1,MathQA_dev,Math,a 3 - digit positive integer is chosen at rand...,b,Step 1: Calculate the total number of 3-digit ...,e,"[(0, 0)]",\nStep 1: Calculate the total number of 3-digi...,e,"[(0, 0)]",...,"[(0, 0)]",Step 1: Understand the problem. We need to fin...,5 / 9,"[(0, 0)]",\nStep 1: Calculate the total number of 3-digi...,e,"[(0, 0)]",\nStep 1: Calculate the total number of 3-digi...,e,"[(0, 0)]"
2,MathQA_dev,Math,if x and y are positive integers and 7 + x + y...,c,\nStep 1: Rearrange the given equation 7 + x +...,c,"[(0, 0)]",\nStep 1: Rearrange the given equation 7 + x +...,a,"[(0, 0)]",...,"[(0, 0)]",\n Step 1: Rearrange the given equation...,e,"[(0, 0)]",\n Step 1: Rearrange the given equation...,c,"[(0, 0)]",\n Step 1: Rearrange the given equation...,c,"[(0, 0)]"
3,MathQA_dev,Math,the hcf and lcm of two numbers m and n are res...,d,"\n To find the values of m and n, we fi...",d,"[(0, 0)]",\n Step 1: Find the relationship betwee...,d,"[(0, 0)]",...,"[(0, 0)]",\n Step 1: Find the relationship betwee...,d,"[(0, 0)]",\n Step 1: Find the relationship betwee...,1/18,"[(0, 0)]",\n Step 1: Find the relationship betwee...,b) 3 / 35,"[(0, 0)]"
4,MathQA_dev,Math,"in a kilometer race , a beats b by 48 meters o...",a,\nStep 1: Calculate the speed difference betwe...,a,"[(0, 0)]",\n Step 1: Calculate the speed differen...,b,"[(0, 0)]",...,"[(0, 0)]",\n Let's denote the speed of A as Va an...,b,"[(0, 0)]",\nStep 1: Calculate the speed difference betwe...,c) 667 sec,"[(0, 0)]",\nStep 1: Calculate the speed difference betwe...,d,"[(0, 0)]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,MathQA_dev,Math,the first doughnut is priced at $ 1 and then i...,b,\n Step 1: Let x be the number of dozen...,b,"[(0, 0)]",Step 1: Let x be the number of dozens purchase...,b,"[(0, 0)]",...,"[(0, 0)]",Step 1: Let x be the number of dozens purchase...,b,"[(0, 0)]",Step 1: Let x be the total number of dozens pu...,b,"[(0, 0)]",\nStep 1: Let's denote the number of individua...,17,"[(0, 0)]"
496,MathQA_dev,Math,"during the first two weeks of january , the to...",e,\nStep 1: Let x be the rainfall during the fir...,e,"[(0, 0)]",Step 1: Let x be the rainfall during the first...,e,"[(0, 0)]",...,"[(0, 0)]",Step 1: Let x be the rainfall during the first...,e,"[(0, 0)]",Step 1: Let x be the rainfall during the first...,e,"[(0, 0)]",\n Step 1: Let x be the rainfall during...,e,"[(0, 0)]"
497,MathQA_dev,Math,a train crosses a bridge of length 200 m in 10...,c,\nStep 1: Calculate the speed of the train by ...,c,"[(1, 0)]",\n Step 1: Calculate the speed of the t...,100 m,"[(0, 0)]",...,"[(0, 0)]",\nStep 1: Let the length of the train be x met...,c,"[(0, 1), (0, 1)]",\n Step 1: Let the length of the train ...,b,"[(0, 0)]",Step 1: Let's denote the length of the train a...,b,"[(0, 0)]"
498,MathQA_dev,Math,10 ^ ( 600 ) ã · 10 ^ ( 580 ) = ? The options ...,b,\n Step 1: Apply the rule of exponents ...,e,"[(0, 1), (0, 1)]",Step 1: Apply the rule of exponents which stat...,10 ^ 1180,"[(0, 1), (0, 1)]",...,"[(0, 1), (0, 1)]",\n Step 1: Apply the rule of exponents ...,e,"[(0, 0)]",Step 1: Apply the rule of exponents which stat...,10 ^ 1180,"[(0, 0)]",\n Step 1: Apply the rule of exponents ...,e,"[(0, 1), (0, 1)]"


In [61]:
import re
def clean_final_answers_gsm8k(column):
    cleaned_answers = []
    for entry in column:
        # Extract numbers using regular expression
        numbers = re.findall(r'\b\d+\b', str(entry))
        if numbers:
            # Join all numbers with space if there are multiple numbers
            cleaned_answers.append(' '.join(numbers))
        else:
            # If no number is found, replace with 'error'
            cleaned_answers.append('error')
    return cleaned_answers
def clean_final_answers_MC(column):
    cleaned_answers = []
    for entry in column:
        # Extract numbers using regular expression
        entry = str(entry).lower()
        options = re.findall(r'[abcde]\s?\)', str(entry))
        if options:
            # Join all numbers with space if there are multiple numbers
            cleaned_answers.append(options[0][0])
        else:
            # If no number is found, replace with 'error'
            cleaned_answers.append(entry)
    return cleaned_answers

In [62]:
for col in df.columns:
    if col.startswith('Final Answer_'):
        df[col] = clean_final_answers_MC(df[col])

# Display the cleaned DataFrame
df_cleaned_updated = df


In [63]:
df_cleaned_updated.to_csv(file_path,index=False)

In [18]:
import numpy as np
from collections import Counter
result_dict = {
    'id':[],
    'correct answer': [],
    'CoT answers':[]
    
}
tmp = []
for col in df_cleaned_updated.columns:
    if col.startswith('Final Answer_'):
        tmp.append(df_cleaned_updated[col].to_numpy())
tmp_arr = np.vstack(tmp)
result_li = tmp_arr.T.tolist()

for row in range(len(df_cleaned_updated)):
    print(df_cleaned_updated.iloc[row]['Correct Answer'])
    print(Counter(result_li[row]))

18
Counter({'18': 40})
3
Counter({'3': 29, '1 5': 6, '45353': 2, '1': 2, '5': 1})
70000
Counter({'195000': 21, '65000': 14, '70000': 3, '52000': 1, '78000': 1})
540
Counter({'540': 40})
20
Counter({'10': 13, '35': 12, '140': 5, '4 21': 2, '20': 2, '3': 1, '760': 1, '8': 1, '0': 1, '40': 1, '60': 1})
64
Counter({'64': 28, '94': 6, '96': 4, '5 3 5 3 5 8 40': 1, '80': 1})
260
Counter({'260': 30, '180': 4, '380': 4, '240': 2})
160
Counter({'120': 16, '160': 12, '260': 5, '100': 3, '60': 2, '90': 1, '180': 1})
45
Counter({'315': 18, '285': 14, '240': 4, '235': 2, '285 4': 1, '290 4': 1})
460
Counter({'460': 38, '520': 2})
366
Counter({'222': 29, '252': 4, '366': 4, '60 60 3 60 3 0 3 60 3 60 180 126 366': 2, '60 60 3 60 3 0 7 60 180 126 366': 1})
694
Counter({'694': 39, '204 160 330 694': 1})
13
Counter({'12': 27, '20': 4, '6': 4, '7': 2, '9': 2, '60': 1})
18
Counter({'18': 31, '42': 3, '24': 2, '21': 1, '39': 1, '33': 1, '30': 1})
60
Counter({'0 6': 39, 'error': 1})
125
Counter({'error': 22

In [30]:
step_count_buffer = []
for col in df_cleaned_updated:
    if col.startswith('CoT_'):
        cleaned_answers = []
        for entry in df_cleaned_updated[col]:
            # Extract numbers using regular expression
            steps = re.findall(r'[Ss]tep\s?\d', str(entry))
            if steps:
                # Join all numbers with space if there are multiple numbers
                cleaned_answers.append(len(steps))
            else:
                # If no number is found, replace with 'error'
                cleaned_answers.append(0)
        step_count_buffer.append(cleaned_answers)

step_count = np.array(step_count_buffer).T.tolist()

(500, 40)